### **Import code**

In [ ]:
%cd ..
from kinetic_model.visualize import GraphSpecBuilder, CytoscapeExporter
import json
import plotly.io as pio
pio.renderers.default = "notebook_connected"

### **Test model visualization**

In [ ]:
# Load the model
with open('modelTemplates/kombucha.json', 'r') as f:
    model = json.load(f)

# Build the graph specification
builder = GraphSpecBuilder()
graph_spec = builder.build_from_json(model)

# Export to Cytoscape
exporter = CytoscapeExporter()
cytoscape_data = exporter.export(graph_spec, layout="nice", show_edge_labels=True)

# Display the visualization
from ipycytoscape import CytoscapeWidget
import ipywidgets as widgets

# Create the viewer
viewer = CytoscapeWidget()
viewer.graph.add_graph_from_json(cytoscape_data['elements'])
viewer.set_style(cytoscape_data['style'])  # Use 'style' instead of 'stylesheet'
viewer.set_layout(name="preset")

# Display
display(viewer)

### **Test model**

In [ ]:
from kinetic_model import ModelFromJson, Pulse, Reactor

def main():
    print("Loading model...")
    model = ModelFromJson("modelTemplates/kombucha_refined.json")

    model_feed = ModelFromJson("modelTemplates/kombucha_refined.json")

    # Optional tweaks:

    #model.microbiome.bacteria["sc"].subpopulations['x1'].count = 4
    #model.microbiome.bacteria["kx"].subpopulations['x4'].count = 4
    #model.microbiome.bacteria["aa"].subpopulations['x7'].count = 4
    #model.microbiome.bacteria["lp"].subpopulations['x10'].count = 4
    #model.microbiome.bacteria["zb"].subpopulations['x12'].count = 4

    # No back-compat property; just check truthiness
    has_env = bool(model.environment)
    print(f"Model loaded with environment: {has_env}")
    if has_env:
        # pH object exposes current pH value
        print(f"Environment pH base: {model.environment.pH.pH}")

    # Single long pulse with continuous flow (qin/qout).
    # Note: we do NOT pass 'continuous_feed_metabolome' (unsupported).
    model.environment.stirring.set_rate(1)
    pulse = Pulse(
        t_start=0.0,
        t_end=500.0,   # 120 hours
        n_steps=100,
        qin=0.0,
        qout=0.0,
        vin=0.1,
        vout=0.1,
        environment=model.environment,
        continuous_feed_metabolome = model_feed.metabolome,
        instant_feed_metabolome = model_feed.metabolome,

    )

    reactor = Reactor(
        microbiome=model.microbiome,
        metabolome=model.metabolome,
        pulses=[pulse],
        volume=1.0,
    )
    reactor.set_fast_simulation_mode()

    print("Starting simulation...")
    reactor.simulate()
    print("Simulation complete!")

    # In a notebook you can visualize:
    # fig = reactor.make_plot()
    # fig

    return reactor

reactor = main()

In [ ]:
reactor.make_plot()